# Ejercicio 1.

_Ejercicio: Entrena un `LinearSVC` en un conjunto de datos linealmente separable. Luego, entrena un `SVC` y un `SGDClassifier` en el mismo conjunto. Comprueba si puedes obtener modelos que sean, aproximadamente, iguales._

Utilizaremos el dataset Iris, ya que las clases Iris Setosa e Iris Versicolor son linealmente separables.

In [ ]:
from sklearn import datasets

iris = datasets.load_iris(as_frame=True)
X = iris.data[["petal length (cm)", "petal width (cm)"]].values
y = iris.target

# Filtramos para quedarnos con Iris setosa e Iris versicolor
setosa_or_versicolor = (y == 0) | (y == 1)
X = X[setosa_or_versicolor]
y = y[setosa_or_versicolor]

Ahora construimos y entrenamos 3 modelos:
- **LinearSVC**: se utiliza con `loss="hinge"` para que sea comparable.
- **SVC**: se especifica `kernel="linear"` para que sea equivalente.
- **SGDClassifier**: se ajusta el parámetro `alpha` para que la regularización sea similar a la de los otros modelos.

In [ ]:
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler

C = 5
alpha = 0.05

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

lin_clf = LinearSVC().fit()
svc_clf = SVC().fit()
sgd_clf = SGDClassifier().fit()

Visualizamos las fronteras de decisión de los tres modelos.

In [ ]:
def compute_decision_boundary(model):
    """
    Calcula dos puntos en el espacio original (desescalado) que definen la línea de decisión.
    """
    w = -model.coef_[0, 0] / model.coef_[0, 1]
    b = -model.intercept_[0] / model.coef_[0, 1]
    return scaler.inverse_transform([[-10, -10 * w + b], [10, 10 * w + b]])

lin_line = compute_decision_boundary()
svc_line = compute_decision_boundary()
sgd_line = compute_decision_boundary()

# Dibujamos las fronteras de decisión de los tres modelos
plt.figure(figsize=(11, 4))
plt.plot(lin_line[:, 0], lin_line[:, 1], "k:", label="LinearSVC")
plt.plot(svc_line[:, 0], svc_line[:, 1], "b--", linewidth=2, label="SVC")
plt.plot(sgd_line[:, 0], sgd_line[:, 1], "r-", label="SGDClassifier")
plt.plot(X[:, 0][y == 1], X[:, 1][y == 1], "bs", label="Iris versicolor")
plt.plot(X[:, 0][y == 0], X[:, 1][y == 0], "yo", label="Iris setosa")
plt.xlabel("Longitud del pétalo")
plt.ylabel("Ancho del pétalo")
plt.legend(loc="upper center")
plt.axis([0, 5.5, 0, 2])
plt.grid()

plt.show()

¡Las fronteras resultantes son bastante similares!

# Ejercicio 10.

_Ejercicio: Entrena un clasificador SVM en el dataset Wine (disponible con `sklearn.datasets.load_wine()`). Este dataset contiene análisis químicos de 178 muestras de vino producidas por 3 cultivares distintos. El objetivo es entrenar un modelo capaz de predecir el cultivador basándose en el análisis químico. Dado que las SVM son clasificadores binarios, se debe usar una estrategia uno-contra-todos. ¿Qué precisión puedes alcanzar?_

In [ ]:
from sklearn.datasets import load_wine

wine = load_wine(as_frame=True)

In [ ]:
print(wine.DESCR)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    wine.data, wine.target, random_state=42)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

Empezamos con un clasificador SVM lineal. Al usar LinearSVC se aplica la estrategia One-vs-Rest (uno-contra-todos) de forma automática.

In [ ]:
lin_clf = LinearSVC(dual=True, random_state=42)
# Entrenamiento sin escalar: es probable que falle la convergencia
try:
    lin_clf.fit(X_train, y_train)
except Exception as e:
    print("Error de convergencia:", e)

Aumentar el número de iteraciones a un millón, el modelo converge?.

In [ ]:
lin_clf = LinearSVC(dual=True, random_state=42)
lin_clf.fit(X_train, y_train)

Evaluamos el modelo con validación cruzada (baseline):


In [ ]:
from sklearn.model_selection import cross_val_score

score_lin = cross_val_score()
print("Precisión sin escalar (LinearSVC):", score_lin)

La precisión obtenida es de aproximadamente 91%, lo cual no es óptimo. Que olvidamos?

Ahora, el modelo converge sin problemas. Evaluamos su rendimiento:

In [ ]:
score_lin_scaled = cross_val_score()
print("Precisión con escalado (LinearSVC):", score_lin_scaled)

Se alcanza una precisión de alrededor del 97.7%, lo cual es muy bueno.

Probamos a continuación un SVM con núcleo (SVC) para ver si mejora:

In [ ]:
score_svc = cross_val_score()
print("Precisión (SVC) sin ajuste:", score_svc)

Aunque no mejora, podemos ajustar los hiperparámetros para obtener mejores resultados:

**NOTA:** _Optimización clases no vistas todavía_

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform, uniform

param_distrib = {
    "svc__gamma": loguniform(0.001, 0.1),
    "svc__C": uniform(1, 10)
}
rnd_search_cv = RandomizedSearchCV(svm_clf, param_distrib, n_iter=100, cv=5, random_state=42)
rnd_search_cv.fit(X_train, y_train)
print("Mejor estimador:", rnd_search_cv.best_estimator_)

In [ ]:
print("Precisión en CV (SVC ajustado):", rnd_search_cv.best_score_)

El modelo SVC ajustado tiene un buen desempeño. Ahora lo evaluamos en el conjunto de prueba:

In [ ]:
test_score = rnd_search_cv.score(X_test, y_test)
print("Precisión en el conjunto de prueba:", test_score)

Aunque el SVC ajustado supera al LinearSVC en validación cruzada, el rendimiento en el conjunto de prueba es un poco menor, lo que puede deberse a un leve sobreajuste durante el ajuste de hiperparámetros. En cualquier caso, el resultado es aceptable.

# Ejercicio 11.

_Ejercicio: Entrena y ajusta un regresor SVM en el dataset de California Housing. Puedes utilizar el dataset original (disponible con `sklearn.datasets.fetch_california_housing()`). Los objetivos representan cientos de miles de dólares. Debido a que hay más de 20,000 instancias, las SVM pueden ser lentas, por lo que para la sintonización de hiperparámetros se recomienda usar menos instancias (por ejemplo, 2,000). ¿Cuál es el RMSE de tu mejor modelo?_

In [ ]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
X = housing.data
y = housing.target

Dividimos el dataset en entrenamiento y prueba:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Recuerda que es fundamental escalar los datos cuando se usan SVM.

Empezamos con un modelo simple de regresión lineal utilizando LinearSVR:

In [ ]:
from sklearn.svm import LinearSVR

lin_svr.fit(X_train, y_train)

Si no converge, incrementamos el número máximo de iteraciones:

In [ ]:
lin_svr.fit(X_train, y_train)

Evaluamos el rendimiento en el conjunto de entrenamiento:

In [ ]:
from sklearn.metrics import root_mean_squared_error

y_pred = lin_svr.predict(X_train)
rmse = root_mean_squared_error(y_train, y_pred)
print("RMSE (LinearSVR):", rmse)

Dado que en este dataset los objetivos representan cientos de miles de dólares, el RMSE indica el error esperado (por ejemplo, errores de ~98,000 dólares no serían sorprendentes). Probemos a mejorar el resultado usando un SVM con núcleo RBF.

Utilizaremos búsqueda aleatoria con validación cruzada para ajustar los hiperparámetros `C` y `gamma`.

**NOTA:** _Optimización no vista aún_

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform, uniform


param_distrib = {
    "svr__gamma": loguniform(0.001, 0.1),
    "svr__C": uniform(1, 10)
}
# Utilizamos solo las primeras 2000 instancias para la sintonización
rnd_search_cv = RandomizedSearchCV(svm_reg, param_distrib, n_iter=100, cv=3, random_state=42)
rnd_search_cv.fit(X_train[:2000], y_train[:2000])

In [ ]:
print("Mejor estimador (SVR ajustado):", rnd_search_cv.best_estimator_)

In [ ]:
# Evaluamos mediante validación cruzada en el conjunto de entrenamiento
cv_scores = -cross_val_score(rnd_search_cv.best_estimator_, X_train, y_train, scoring="neg_root_mean_squared_error")
print("RMSE CV (SVR ajustado):", cv_scores.mean())

El modelo ajustado con núcleo RBF muestra un RMSE considerablemente menor en validación cruzada.

Ahora lo evaluamos en el conjunto de prueba:

In [ ]:
y_pred = rnd_search_cv.best_estimator_.predict(X_test)
rmse_test = root_mean_squared_error(y_test, y_pred, squared=False)
print("RMSE en el conjunto de prueba (SVR ajustado):", rmse_test)

En este caso, las SVM funcionan muy bien para el dataset Wine, pero en el dataset California Housing otros métodos (por ejemplo, Random Forests) pueden superar su rendimiento.

¡Y eso es todo por hoy!